In [2]:
import pandas as pd
import numpy as np
import datetime 
import math as mat
import time 
from datetime import timedelta
import matplotlib.pyplot as plt

In [3]:
accre_df=pd.read_csv('Data/july-2017-thru-june-2018.txt',sep="|",nrows=1000000)


In [6]:
accre_df.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,COMPLETED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21


In [7]:
accre_df['Elapsed'].head()

0    1-01:05:21
1      23:51:47
2    1-01:59:51
3    1-08:02:20
4      00:00:10
Name: Elapsed, dtype: object

### calculate the duration of a job
### job_duration=end_time-start_time

In [8]:
start_time=accre_df['Start']
end_time=accre_df['End']
submit_time=accre_df['Submit']

In [9]:
print(start_time.head());print(end_time.head())

0    2017-07-03T02:13:59
1    2017-07-03T02:20:08
2    2017-07-03T02:41:13
3    2017-07-03T02:51:53
4    2017-07-01T05:00:01
Name: Start, dtype: object
0    2017-07-04T03:19:20
1    2017-07-04T02:11:55
2    2017-07-04T04:41:04
3    2017-07-04T10:54:13
4    2017-07-01T05:00:11
Name: End, dtype: object


# convert the start and end time from str to datetime format to calculate job duration

In [10]:
submit_time=pd.to_datetime(submit_time)

In [11]:
start_time=pd.to_datetime(start_time)

In [12]:
end_time=pd.to_datetime(end_time)

# calculate duration of a job

In [13]:
job_duration=(end_time-start_time)

In [14]:
job_duration=pd.DataFrame(job_duration)

In [15]:
job_duration.head()

,0
0,1 days 01:05:21
1,0 days 23:51:47
2,1 days 01:59:51
3,1 days 08:02:20
4,0 days 00:00:10


In [16]:
job_duration.shape

(1000000, 1)

In [17]:
accre_df.shape

(1000000, 25)

#  join the job_duration with accre_df

In [18]:

accre_df2=accre_df.join(job_duration)

In [19]:
accre_df2.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,0
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...,1 days 01:05:21
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...,0 days 23:51:47
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...,1 days 01:59:51
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...,1 days 08:02:20
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21,0 days 00:00:10



# rename the column of job_duration 

In [20]:

accre_df2.rename(columns={0:'job_duration'},inplace=True)

In [21]:
#accre_df2=accre_df2.set_index('Submit')
# accre_df2.resample('4h').count()

In [22]:
accre_df2.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,job_duration
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...,1 days 01:05:21
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...,0 days 23:51:47
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...,1 days 01:59:51
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...,1 days 08:02:20
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21,0 days 00:00:10


In [23]:
# make it dataframe
submit_time=pd.DataFrame(submit_time)

In [76]:
# replace the string value for sumbit time with datetime formated Submit_time
accre_df2['Submit']=submit_time

In [25]:
accre_df2.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,job_duration
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02 00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...,1 days 01:05:21
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02 00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...,0 days 23:51:47
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02 00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...,1 days 01:59:51
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02 00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...,1 days 08:02:20
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01 05:00:00,...,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21,0 days 00:00:10


In [26]:
accre_df2['Submit'].head()

0   2017-07-02 00:40:58
1   2017-07-02 00:40:58
2   2017-07-02 00:40:58
3   2017-07-02 00:40:58
4   2017-07-01 05:00:00
Name: Submit, dtype: datetime64[ns]

# check data type of job_duration column

In [27]:
accre_df2['job_duration'].head()

0   1 days 01:05:21
1   0 days 23:51:47
2   1 days 01:59:51
3   1 days 08:02:20
4   0 days 00:00:10
Name: job_duration, dtype: timedelta64[ns]

In [205]:
accre_select_df=accre_df2.loc[:,['User','JobID','Submit','Start','job_duration']]
accre_select_df.head()


,User,JobID,Submit,Start,job_duration
0,masispider,15806649,2017-07-02 00:40:58,2017-07-03T02:13:59,1 days 01:05:21
1,masispider,15806664,2017-07-02 00:40:58,2017-07-03T02:20:08,0 days 23:51:47
2,masispider,15806670,2017-07-02 00:40:58,2017-07-03T02:41:13,1 days 01:59:51
3,masispider,15806671,2017-07-02 00:40:58,2017-07-03T02:51:53,1 days 08:02:20
4,uscms010,15811617,2017-07-01 05:00:00,2017-07-01T05:00:01,0 days 00:00:10


In [206]:
accre_select_df=accre_select_df.set_index('Submit')
accre_select_df.head()

,User,JobID,Start,job_duration
Submit,,,,
2017-07-02 00:40:58,masispider,15806649,2017-07-03T02:13:59,1 days 01:05:21
2017-07-02 00:40:58,masispider,15806664,2017-07-03T02:20:08,0 days 23:51:47
2017-07-02 00:40:58,masispider,15806670,2017-07-03T02:41:13,1 days 01:59:51
2017-07-02 00:40:58,masispider,15806671,2017-07-03T02:51:53,1 days 08:02:20
2017-07-01 05:00:00,uscms010,15811617,2017-07-01T05:00:01,0 days 00:00:10


# sort the the accre_select_df by job_duration and user name in ascending order.

In [209]:
accre_select_df=accre_select_df.sort_values(by=['Submit','User','Start','job_duration'],ascending=True)

In [211]:
accre_select_df.head()

,User,JobID,Start,job_duration
Submit,,,,
2017-07-01 05:00:00,uscms010,15811617,2017-07-01T05:00:01,00:00:10
2017-07-01 05:00:01,autocms,15811618,2017-07-01T05:00:04,00:03:18
2017-07-01 05:00:02,tuos,15811619,2017-07-01T05:00:04,00:02:03
2017-07-01 05:00:22,uscms010,15811621,2017-07-01T05:00:23,00:00:10
2017-07-01 05:01:01,autocms,15811624,2017-07-01T05:01:02,00:04:47


In [186]:
# accre_select_df=accre_select_df.set_index(['User','Submit'])

In [237]:
#accre_select_df=accre_select_df.set_index(['Submit','User'])

In [212]:
accre_select_df.head()

,User,JobID,Start,job_duration
Submit,,,,
2017-07-01 05:00:00,uscms010,15811617,2017-07-01T05:00:01,00:00:10
2017-07-01 05:00:01,autocms,15811618,2017-07-01T05:00:04,00:03:18
2017-07-01 05:00:02,tuos,15811619,2017-07-01T05:00:04,00:02:03
2017-07-01 05:00:22,uscms010,15811621,2017-07-01T05:00:23,00:00:10
2017-07-01 05:01:01,autocms,15811624,2017-07-01T05:01:02,00:04:47


In [213]:
accre_5_minutes= accre_select_df[accre_select_df.job_duration<'0 days 00:05:00']

In [214]:
accre_5_minutes.head()

,User,JobID,Start,job_duration
Submit,,,,
2017-07-01 05:00:00,uscms010,15811617,2017-07-01T05:00:01,00:00:10
2017-07-01 05:00:01,autocms,15811618,2017-07-01T05:00:04,00:03:18
2017-07-01 05:00:02,tuos,15811619,2017-07-01T05:00:04,00:02:03
2017-07-01 05:00:22,uscms010,15811621,2017-07-01T05:00:23,00:00:10
2017-07-01 05:01:01,autocms,15811624,2017-07-01T05:01:02,00:04:47


level_values = accre_5_minutes.index.get_level_values
result = (accre_5_minutes.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='4h', level=-1)]).count())

result.head(50)

In [215]:
#accre_5_minutes.sort_index(inplace=True)
#accre_5_minutes.head(20)

In [219]:
accre_5_mnts_grouped=accre_5_minutes.groupby(['Submit','User']).count()

In [220]:
accre_5_mnts_grouped.head()

,,JobID,Start,job_duration
Submit,User,,,
2017-07-01 05:00:00,uscms010,1,1,1
2017-07-01 05:00:01,autocms,1,1,1
2017-07-01 05:00:02,tuos,1,1,1
2017-07-01 05:00:22,uscms010,1,1,1
2017-07-01 05:01:01,autocms,1,1,1


In [221]:
accre_5_mnts_reindexed=accre_5_mnts_grouped.reset_index('User')

In [223]:
accre_5_mnts_reindexed.head()

,User,JobID,Start,job_duration
Submit,,,,
2017-07-01 05:00:00,uscms010,1,1,1
2017-07-01 05:00:01,autocms,1,1,1
2017-07-01 05:00:02,tuos,1,1,1
2017-07-01 05:00:22,uscms010,1,1,1
2017-07-01 05:01:01,autocms,1,1,1


In [224]:
del(accre_5_mnts_reindexed['Start'])

In [225]:
del(accre_5_mnts_reindexed['job_duration'])

# users running more than 500 jobs shorter than 5 minutes.

In [226]:

accre_5_mnts_reindexed[accre_5_mnts_reindexed.JobID>500]

accre_5_mnts_reindexed.head(10)

,User,JobID
Submit,,
2017-07-01 05:00:00,uscms010,1
2017-07-01 05:00:01,autocms,1
2017-07-01 05:00:02,tuos,1
2017-07-01 05:00:22,uscms010,1
2017-07-01 05:01:01,autocms,1
2017-07-01 05:01:02,tuos,1
2017-07-01 05:01:26,vuiiscci,2
2017-07-01 05:01:56,uscms010,1
2017-07-01 05:02:02,autocms,1


In [239]:
# accre_5_mnts_reindexed.rolling('4H').sum()

In [229]:
#accre_5_mnts_reindexed.reset_index(level=0, inplace =True)
#accre_5_mnts_reindexed.head(10)

In [139]:
#accre_5_mnts_reindexed.rolling('4H',on='Submit').sum()

#accre_5_mnts_reindexed.shift(4,axis=0)

In [230]:
#accre_5_mnts_reindexed.resample('4h').sum()

#accre_5_mnts_reindexed.head()

for date in accre_5_mnts_reindexed['Submit']:
    list =[]
    count = 1
    dateindex = 0
    if pd.Timedelta(Submit[1]-Submit[0]).seconds/3600 < 4
        dateindex = dateindex + 1
        count = count + JobID
    elif pd.Timedelta(Submit[1]-Submit[0]).seconds/3600 >= 4
        list.append(count)
        
        

In [105]:
accre_5_mnts_reindexed['Submit'].dtype

dtype('<M8[ns]')

In [104]:
# accre_5_mnts_reindexed.rolling().sum()

In [ ]:
for x in accre_5_mnts_reindexed.Submit:
    accre_5_mnts_reindexed

In [57]:
# grouped.resample('4h').count()

In [435]:
#accre_5_minutes.resample('4h','JobID').count()

In [320]:
# accre_5_minutes.resample('4h').count()

In [321]:
# accre_5_mnts_per_4h.head()

In [323]:
# df.reset_index(level=0, inplace=True)

DataFrame.resample(rule, how=None, axis=0, fill_method=None, closed=None, label=None, convention='start', kind=None, loffset=None, limit=None, base=0, on=None, level=None)

In [233]:
#count_5_minutes_grouped=accre_5_minutes.groupby(['User','Submit'])['JobID'].count()

In [232]:
#count_5_minutes_grouped.resample('4h').count()

In [234]:
#count_5_minutes_grouped.head()

# users sumbitted more than 500 jobs each taking shorter than 5 minutes grouped by month,date and hour

In [236]:
## counts per months, day and hour
##count_5_minutes_job[count_5_minutes_job> 500]

# some notes from Mary
# to get random 5000 rows of jobs

indices=np.random.randint(0,len(jobs),5000)
jobs.subset=jobs.iloc[indices]
#1.  most frequent values
jobs.ReqMem.valuecounts(ascending=False).head(5000)
 # this returns df
#2. from collections import Counter

import Counter from collections
mem.types=Counter(jobs.RegMem)
mem.typyes.most_common(5000)    

# returns a dictionary like counter object.
    

In [59]:

# elapsed time is the same as duration

# accre_df.loc[:,['JobID','Elapsed']].head()